In [3]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
df = pd.read_csv(file_to_load)


In [213]:
#Pandas Dataframe head command list the first 5 rows by default with column names shown.
df.head() 
#df.tail()
#df.describe()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [5]:
n_player=len(df.SN.unique())
print("total number of players = ", n_player)

total number of players =  576


In [206]:
# 2. Purchase Analysis (Total)
#      -- Number of Unique Items
#      -- Average Purchase Price
#      -- Total Number of Purchases
#      -- Total Revenue

pd.DataFrame ({
   "Number of Unique_Items":len(df["Item ID"].unique()), 
     "average purchase price":df.Price.mean(),
     "Total Number of Purchases":df["Purchase ID"].count(),
     "Total Revenue":df.Price.sum()
}, index = [0])



,Number of Unique_Items,average purchase price,Total Number of Purchases,Total Revenue
0,179,3.050987,780,2379.77


In [7]:
#   3. Gender Demographics -- Percentage and Count of Male Players
#                         -- Percentage and Count of Female Players
#                         -- Percentage and Count of Other / Non-Disclosed


#df.Gender.value_counts(normalize = True)

pd.DataFrame ({
    "Percentage":df.Gender.value_counts(normalize = True).mul(100).round(2), 
    "Count":df.Gender.value_counts()
})

,Percentage,Count
Male,83.59,652
Female,14.49,113
Other / Non-Disclosed,1.92,15


In [207]:
#   4. Purchasing Analysis (By Gender) -- Purchase Count
#                                  -- Average Purchase Price
#                                  -- Total Purchase Price
#                                  -- Average Purchase total per Person by Gender

# groups = df.groupby(["Gender"])
# print(groups.Price.describe())

print('\nPurcase Count:')
purchase_count = df.groupby('Gender').count()['Purchase ID']
print(purchase_count)

print('\nAverage Purchase Price:')
mean_purchase_price = df.groupby('Gender').mean()['Price']
print(mean_purchase_price)

print('\nTotal Purchase Price:')
total_price = df.groupby('Gender').sum()['Price']
print(total_price)

print('\nAverage Purchase Total per Person by Gender:')
mean_purchase_total = df.groupby('Gender').apply(lambda x: x.groupby(["SN"]).Price.sum().mean())
print(mean_purchase_total)



Purcase Count:
Gender
Female                   113
Male                     652
Other / Non-Disclosed     15
Name: Purchase ID, dtype: int64

Average Purchase Price:
Gender
Female                   3.203009
Male                     3.017853
Other / Non-Disclosed    3.346000
Name: Price, dtype: float64

Total Purchase Price:
Gender
Female                    361.94
Male                     1967.64
Other / Non-Disclosed      50.19
Name: Price, dtype: float64

Average Purchase Total per Person by Gender:
Gender
Female                   4.468395
Male                     4.065372
Other / Non-Disclosed    4.562727
dtype: float64


In [208]:
#    5. 
#       Age Demographics
#       Purchase Count, Average Purchase Price, Total Purchase Value, Average Purchase total per person by Age Group

#    print(df.Age.min()) = 7
#    print(df.Age.max()) = 45

#    Establish the bins of 4 years
age_bins = [0, 9, 14, 19, 23, 27, 31, 36, 41, 46]
group_names = ["0-9", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40-45"]
groups = df.groupby(pd.cut(df.Age, age_bins))

print('\nPurcase Count:')
print(groups.count()['Purchase ID'])

print('\nAverage Purchase Price:')
print(groups.mean()['Price'])

print('\nTotal Purchase Price:')
print(groups.sum()['Price'])

print('\nAverage Purchase Total per Person by Age Group:')

mean_purchase_total = groups.apply(lambda x: x.groupby(["SN"]).Price.sum().mean())
print(mean_purchase_total)



Purcase Count:
Age
(0, 9]       23
(9, 14]      28
(14, 19]    136
(19, 23]    298
(23, 27]    150
(27, 31]     60
(31, 36]     50
(36, 41]     30
(41, 46]      5
Name: Purchase ID, dtype: int64

Average Purchase Price:
Age
(0, 9]      3.353478
(9, 14]     2.956429
(14, 19]    3.035956
(19, 23]    3.033020
(23, 27]    3.063600
(27, 31]    2.967500
(31, 36]    2.886400
(36, 41]    3.541000
(41, 46]    2.998000
Name: Price, dtype: float64

Total Purchase Price:
Age
(0, 9]       77.13
(9, 14]      82.78
(14, 19]    412.89
(19, 23]    903.84
(23, 27]    459.54
(27, 31]    178.05
(31, 36]    144.32
(36, 41]    106.23
(41, 46]     14.99
Name: Price, dtype: float64

Average Purchase Total per Person by Age Group:
Age
(0, 9]      4.537059
(9, 14]     3.762727
(14, 19]    3.858785
(19, 23]    4.304000
(23, 27]    4.140000
(27, 31]    4.046591
(31, 36]    3.900541
(36, 41]    4.618696
(41, 46]    2.998000
dtype: float64


In [64]:
#   6. 
#      Identify the 5 Top Spenders in game by total purchase value and list in a table
#                                 --SN
#                                 --Purchase Count
#                                 --Average Purchase Price
#                                 --Total Purchase Value

groups = df.groupby('SN')
top_5_spenders = groups.Price.sum().sort_values(ascending = False)[:5].index

top_spenders_metrics_dict = {
    "SN":[],
    "Count":[],
    "Ave_Purchase_Price":[],
    "Total_Purchase_Price":[]
}

top_spenders_metrics = pd.DataFrame(top_spenders_metrics_dict)

for group_index in top_5_spenders:
    spender = groups.get_group(group_index)
    
    SN = spender.SN.unique()[0]
    count = spender.Price.count()
    ave_purchase_price = spender.Price.mean()
    total_purchase_price = spender.Price.sum()
    
    row = [{"SN":SN, "Count":count, "Ave_Purchase_Price":ave_purchase_price, "Total_Purchase_Price":total_purchase_price}]
    
    top_spenders_metrics = top_spenders_metrics.append(row) 
    
print (top_spenders_metrics)

   

            SN  Count  Ave_Purchase_Price  Total_Purchase_Price
0    Lisosia93    5.0            3.792000                 18.96
0  Idastidru52    4.0            3.862500                 15.45
0   Chamjask73    3.0            4.610000                 13.83
0       Iral74    4.0            3.405000                 13.62
0  Iskadarya95    3.0            4.366667                 13.10


In [183]:
#   7. 
#       The top 5 Most Popular Items by purchase count and list in a table
#                                 --Item Id
#                                 --Item Name
#                                 --Purchase Count
#                                 --Item Price
#                                 --Total Purchase Value

groups = df.groupby('Item ID')
top_items = groups.Price.count().sort_values(ascending = False)[:5].index

top_items_metrics_dict = {
    "Item ID":[],
    "Item Name":[],
}

top_items_metrics = pd.DataFrame(top_items_metrics_dict)

for group_index in top_items:
    item = groups.get_group(group_index)
    
    row = [{
        "Item ID": item["Item ID"].unique()[0],
        "Item Name": item["Item Name"].unique()[0],
        "Purchase Count": item["Item ID"].count(),
        "Item Price": item["Price"].unique()[0],
        "Total Purchase Value": item["Price"].sum()
     }]
    
    top_items_metrics = top_items_metrics.append(row) 
    
print (top_items_metrics)


   Item ID                                     Item Name  Purchase Count  \
0     92.0                                  Final Critic            13.0   
0    178.0  Oathbreaker, Last Hope of the Breaking Storm            12.0   
0    108.0     Extraction, Quickblade Of Trembling Hands             9.0   
0    132.0                                    Persuasion             9.0   
0     82.0                                       Nirvana             9.0   

   Item Price  Total Purchase Value  
0        4.88                 59.99  
0        4.23                 50.76  
0        3.53                 31.77  
0        3.19                 28.99  
0        4.90                 44.10  


In [197]:
#   8.  
#           5 Most Profitable Items by total purchase value and list in a table
#                                 --Item Id
#                                 --Item Name
#                                 --Purchase Count
#                                 --Item Price
#                                 --Total Purchase Value

groups = df.groupby('Item ID')
profitable_items = groups.Price.sum().sort_values(ascending = False)[:5].index

profitable_items_metrics_dict = {
    "Item ID":[],
    "Item Name":[],
    "Purchase Count":[],
}

profitable_items_metrics = pd.DataFrame(profitable_items_metrics_dict)

for item in profitable_items:
    profitable_item = groups.get_group(item)
       
    row = [{
        "Item ID": profitable_item["Item ID"].unique()[0],
        "Item Name": profitable_item["Item Name"].unique()[0],
        "Purchase Count": profitable_item["Item ID"].count(),
        "Item Price": profitable_item["Price"].unique()[0],
        "Total Purchase Value": profitable_item["Price"].sum()            
     }]
    
    profitable_items_metrics = profitable_items_metrics.append(row)
print (profitable_items_metrics)




   Item ID                                     Item Name  Purchase Count  \
0     92.0                                  Final Critic            13.0   
0    178.0  Oathbreaker, Last Hope of the Breaking Storm            12.0   
0     82.0                                       Nirvana             9.0   
0    145.0                          Fiery Glass Crusader             9.0   
0    103.0                                Singed Scalpel             8.0   

   Item Price  Total Purchase Value  
0        4.88                 59.99  
0        4.23                 50.76  
0        4.90                 44.10  
0        4.58                 41.22  
0        4.35                 34.80  
